#  The Battle of Neighborhoods


**Overview :**
I will be playing a role of a Consultant who helps people in Restaurant business by  
Recommending appropriate location and cuisine type for the Restaurant .
This analysis is likely to increase the profit margin to a great extent by providing in depth analysis and providing answer to the most important questions :
-	Where to open a new Restaurant
-	Which type of Cuisine is best suitable for the business?

**Problem Statement:**
A Restaurant Company “XYZ” is looking to open new restaurants in Toronto area. They are not able to narrow down the location options and suitable cuisine for the restaurants. The company wants to find out the most suitable location for the restaurant which is likely to get maximum customers possible.  Along with the location, the company also wants to find out the best suitable Cuisine according to the locality which will enhance the chances of the attracting more customers.
The Company have below cuisine options for the restaurant:
-	Vegetarian / Vegan restaurant
-	Mexican restaurant
-	Indian restaurant
-	Japanese restaurant
-	Italian restaurant
**Background:**
Two years back the Restaurant company opened a restaurant in London.
The restaurant business was fine for first two months but soon the business started going down with very huge loss. Eventually the Company decided to close the restaurant due to the Campany had to incur a huge loss.
Keeping the incident in mind, the officials at the Restaurant company decided to get professional consultation for the detailed analysis of where and what before opening a new restaurant chain in Toronto.

**Data Requirement:**

This project will require below data:
-	Postal Code, Borough, and Neighbourhood
-	Geographical coordinates of the Neighbourhood
-	Venues list
-	Restaurant’s preference of cuisine and location (if any).


**Solution:**
-	First the Postal code data will be taken from a web page using Beautifulsoup.
-	Get the geospatial data using the geocoder.
-	Get the Venue related data using the Foursquare API.
-	Analyse and Process the data to make it suitable for use.
-	Implement clustering on the data.
-	Visualize the data using charts, maps and tables get accurate analysis.
-	Get the best possible combination of Location and cuisine for the Restaurant.

Import Pandas and numpy

In [1]:
import pandas as pd
import numpy as np

### Get the data from wikipedia page and create dataframe


### Importing BeautifulSoup and requests

In [2]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

Creating a variable "url" to store the wikipedia link

In [3]:
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"   ---- Not working , Data has been updated on the wiki
# Using old version of the url with data in the correct required format
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=1012023397'

Using padas read_html function to read the url data

In [4]:
data = pd.read_html(url, flavor='bs4')

Create dataframe df and input first table from the data

In [5]:
df=data[0]

In [6]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [7]:
df.shape

(180, 3)

### Removing the rows in which Borough is Not assigned

In [8]:
df = df[df["Borough"] != "Not assigned"]

In [9]:
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
print('Shape: ', df.shape)
print('Check if any assigned value present for Borough: ', df.Neighbourhood.str.count("Not assigned").sum())

Shape:  (103, 3)
Check if any assigned value present for Borough:  0


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough**

**No NaN value present in Neighbourhood**

In [11]:
df.Neighbourhood.str.count("Not assigned").sum()

0

In [136]:
df["Postal Code"].count()

103

In [137]:
df["Postal Code"].nunique()

103

**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table**

In [12]:
df = df.groupby(['Postal Code','Borough']).Neighbourhood.apply(lambda x: ', '.join(x)).to_frame()
df.reset_index(inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Check the shape of our final dataframe

In [13]:
print('Share of Dataframe df: ', df.shape)

Share of Dataframe df:  (103, 3)


**Import Geocoder**

In [15]:
import geocoder # import geocoder

In [16]:
# Read and store the csv from the link provided:

data = pd.read_csv("https://cocl.us/Geospatial_data")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Check the shape of postal code dataframe and geospatial dataframe

Both are same in shape

In [17]:
print("The shape of Postal Code Dataframe: ", df.shape)
print("the shape of Geospatial csv Dataframe: ", data.shape)

The shape of Postal Code Dataframe:  (103, 3)
the shape of Geospatial csv Dataframe:  (103, 3)


In [18]:
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

**Merge the two dataframes**

In [19]:
merged_df = df.join(data.set_index('Postal Code'), on='Postal Code', how='inner')
merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


**Shape of the merged dataset**

In [20]:
print('The shape of the merged dataset: ', merged_df.shape)

The shape of the merged dataset:  (103, 5)


## Importing geopy and folium

In [21]:
# Uncomment this if Geopy is not installed
#!pip install geopy
from geopy.geocoders import Nominatim

In [23]:
# Uncomment this if Folium is not installed
#!pip install folium
import folium

Find the coordinates of the Toronto:

Use the nominatim , pass the required paramters and get the coordinates

In [24]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto_exploration')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto :  {}, {}.'
      .format(latitude, longitude))

The coordinates of Toronto :  43.6534817, -79.3839347.


Creating the map centered in Toronto with the neighborhood marked:

In [25]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add boroughs as markers
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Creating the details required for Foursquare:

In [26]:
CLIENT_ID = 'XKIJ30QKHDEHRRH0IS0TOZJLEJZ10FY0Z0ZMTR00HBT2XKJC' 
CLIENT_SECRET = 'OJ033FI1K5ICVQAHPEJBLC0USERCDVYUCW5AQDDSWZUIIPVU'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XKIJ30QKHDEHRRH0IS0TOZJLEJZ10FY0Z0ZMTR00HBT2XKJC
CLIENT_SECRET:OJ033FI1K5ICVQAHPEJBLC0USERCDVYUCW5AQDDSWZUIIPVU


Function to extract category of the venue

In [27]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Function to get nearby venues

In [28]:
# Function that gets 100 closest venues to a postal code coordinates
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=800):
    
    LIMIT = 100
    venues_list=[]
    
    for code, lat, lng in zip(postal_codes, latitudes, longitudes):
        print('Querying venues from: ', code)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Group Latitude', 
                  'Neighborhood Group Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [29]:
filtered_df = merged_df[merged_df.Borough.str.contains('Toronto',case=False)]
filtered_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [31]:
all_venues = getNearbyVenues(filtered_df['Postal Code'],
                             filtered_df['Latitude'],
                             filtered_df['Longitude']
                            )


print('Data shape: ', all_venues.shape)
all_venues.head()

Querying venues from:  M4E
Querying venues from:  M4K
Querying venues from:  M4L
Querying venues from:  M4M
Querying venues from:  M4N
Querying venues from:  M4P
Querying venues from:  M4R
Querying venues from:  M4S
Querying venues from:  M4T
Querying venues from:  M4V
Querying venues from:  M4W
Querying venues from:  M4X
Querying venues from:  M4Y
Querying venues from:  M5A
Querying venues from:  M5B
Querying venues from:  M5C
Querying venues from:  M5E
Querying venues from:  M5G
Querying venues from:  M5H
Querying venues from:  M5J
Querying venues from:  M5K
Querying venues from:  M5L
Querying venues from:  M5N
Querying venues from:  M5P
Querying venues from:  M5R
Querying venues from:  M5S
Querying venues from:  M5T
Querying venues from:  M5V
Querying venues from:  M5W
Querying venues from:  M5X
Querying venues from:  M6G
Querying venues from:  M6H
Querying venues from:  M6J
Querying venues from:  M6K
Querying venues from:  M6N
Querying venues from:  M6P
Querying venues from:  M6R
Q

,Postal Code,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,M4E,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,M4E,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,M4E,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [32]:
all_venues.groupby('Postal Code').count()

,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,47,47,47,47,47,47
M4K,100,100,100,100,100,100
M4L,51,51,51,51,51,51
M4M,93,93,93,93,93,93
M4N,7,7,7,7,7,7
M4P,56,56,56,56,56,56
M4R,37,37,37,37,37,37
M4S,78,78,78,78,78,78
M4T,13,13,13,13,13,13


**Unique Categories**

In [33]:
print('Unique categories.',format(len(all_venues['Venue Category'].unique())))

Unique categories. 285


Applying One-Hot Encoding for venuw category

In [34]:
# Applying OH Encoding by 'creating dummies'
venues_one_hot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# Add the 'Postal Code' column back to dataset
venues_one_hot['Postal Code'] = all_venues['Postal Code'] 

# Move postal code column to the first column
fixed_columns = [venues_one_hot.columns[-1]] + list(venues_one_hot.columns[:-1])
venues_one_hot = venues_one_hot[fixed_columns]

venues_one_hot.head()

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by Postal code and by taking the mean of the frequency of occurrence of each category:

In [35]:
venues_grouped = venues_one_hot.groupby('Postal Code').mean().reset_index()

venues_grouped

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.021277,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,M4K,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000
2,M4L,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.019608,0.000000
3,M4M,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,...,0.00,0.000000,0.00000,0.000000,0.021505,0.00,0.000000,0.000000,0.000000,0.021505
4,M4N,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,M4P,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017857,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000
6,M4R,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.027027
7,M4S,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.012821,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012821
8,M4T,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,M4V,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,...,0.00,0.016393,0.00000,0.000000,0.016393,0.00,0.000000,0.000000,0.000000,0.016393


The 5 more popular venue categories from each neighborhood group:

# Restaurant Recommender Project

### Unique Categories

In [36]:
all_venues['Venue Category'].unique()

array(['Trail', 'Vegetarian / Vegan Restaurant', 'Bakery', 'Gastropub',
       'Indie Movie Theater', 'Ice Cream Shop', 'Bagel Shop',
       'Health Food Store', 'Toy / Game Store', 'Coffee Shop',
       'French Restaurant', 'Pub', 'Breakfast Spot', 'Mexican Restaurant',
       'Juice Bar', 'Japanese Restaurant', 'Bar',
       'Mediterranean Restaurant', 'Tea Room', 'Indian Restaurant',
       'Thai Restaurant', 'Caribbean Restaurant', 'Sandwich Place',
       'Ramen Restaurant', 'Grocery Store', 'Chocolate Shop', 'Pharmacy',
       'Greek Restaurant', 'Café', 'Asian Restaurant', 'Pizza Place',
       'Bank', 'Neighborhood', 'Electronics Store', 'Nail Salon',
       'Flower Shop', 'Church', 'Beach', 'Cosmetics Shop',
       'Italian Restaurant', 'Yoga Studio', 'Fruit & Vegetable Store',
       'Brewery', 'Bookstore', 'Restaurant', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Concert Hall', 'Spa', 'Park',
       'Tapas Restaurant', 'Cocktail Bar', 'Dessert Shop', 'BBQ Joint',
  

### Unique Restaurants

In [38]:
all_venues[all_venues['Venue Category'].str.contains("Restaurant")]['Venue Category'].unique()

array(['Vegetarian / Vegan Restaurant', 'French Restaurant',
       'Mexican Restaurant', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Indian Restaurant', 'Thai Restaurant',
       'Caribbean Restaurant', 'Ramen Restaurant', 'Greek Restaurant',
       'Asian Restaurant', 'Italian Restaurant', 'Restaurant',
       'Tapas Restaurant', 'Tibetan Restaurant', 'Cuban Restaurant',
       'New American Restaurant', 'Fast Food Restaurant',
       'Falafel Restaurant', 'Chinese Restaurant', 'Sushi Restaurant',
       'Pakistani Restaurant', 'Indian Chinese Restaurant',
       'Seafood Restaurant', 'Middle Eastern Restaurant',
       'American Restaurant', 'Comfort Food Restaurant',
       'Latin American Restaurant', 'Vietnamese Restaurant',
       'Indonesian Restaurant', 'Syrian Restaurant',
       'Cantonese Restaurant', 'German Restaurant',
       'Modern European Restaurant', 'Taiwanese Restaurant',
       'Filipino Restaurant', 'Theme Restaurant', 'Korean Restaurant',
       

#### Now we will take each an every restaurant type to check for the best posible combination.
#### We will cluster the data for each restaurant type and analyse each and every clusters to find and recommend the most suitable Neighborhood for the required Restaurant type.

## Vegan/Vegetarian Restaurants

In [41]:
venues_grouped[venues_grouped["Vegetarian / Vegan Restaurant"] > 0]

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.021277,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000
5,M4P,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.017857,0.0,0.00,0.000000,0.00,0.017857,0.0,0.0,0.000000
7,M4S,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.012821,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.012821
9,M4V,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,...,0.00,0.016393,0.0,0.00,0.016393,0.00,0.000000,0.0,0.0,0.016393
15,M5C,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.00,0.010000,0.0,0.00,0.000000,0.00,0.010000,0.0,0.0,0.000000
16,M5E,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.010000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000
17,M5G,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.01,0.010000,0.0,0.00,0.010000,0.00,0.000000,0.0,0.0,0.010000
18,M5H,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,...,0.01,0.010000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.010000
19,M5J,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.010000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.010000
20,M5K,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.00,0.020000,0.0,0.00,0.000000,0.00,0.010000,0.0,0.0,0.000000


In [42]:
Veg_venues_grouped = venues_grouped[["Postal Code","Vegetarian / Vegan Restaurant"]]

In [43]:
Veg_venues_grouped.head()

,Postal Code,Vegetarian / Vegan Restaurant
0,M4E,0.021277
1,M4K,0.000000
2,M4L,0.000000
3,M4M,0.000000
4,M4N,0.000000


### Clustering

In [44]:
from sklearn.cluster import KMeans
n = 5

Veg_cluster = Veg_venues_grouped.drop(["Postal Code"], 1)

# run k-means clustering
Veg_kmeans = KMeans(n_clusters=n, random_state=1)
Veg_kmeans.fit_transform(Veg_cluster)

# check cluster labels generated for each row in the dataframe
Veg_kmeans.labels_[0:20]

array([1, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3])

In [46]:
Veg_merged = Veg_venues_grouped.copy()

# add clustering labels
Veg_merged["Cluster Labels"] = Veg_kmeans.labels_

In [47]:
Veg_merged.count()

Postal Code                      40
Vegetarian / Vegan Restaurant    40
Cluster Labels                   40
dtype: int64

In [50]:
Veg_merged

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels
0,M4E,0.021277,1
1,M4K,0.000000,0
2,M4L,0.000000,0
3,M4M,0.000000,0
4,M4N,0.000000,0
5,M4P,0.017857,1
6,M4R,0.000000,0
7,M4S,0.012821,3
8,M4T,0.000000,0
9,M4V,0.016393,1


In [51]:
Veg_merged = Veg_merged.join(all_venues.set_index('Postal Code'), on='Postal Code')

print(Veg_merged.shape)
Veg_merged.head()

(2819, 9)


,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,0.021277,1,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,M4E,0.021277,1,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,M4E,0.021277,1,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,M4E,0.021277,1,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,M4E,0.021277,1,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [52]:
Veg_merged.sort_values(["Cluster Labels"], inplace=True)
Veg_merged.head()

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
39,M7Y,0.0,0,43.662744,-79.321558,Lakeshore Blvd E & Leslie St,43.658763,-79.328988,Intersection
12,M4Y,0.0,0,43.665860,-79.383160,H Mart,43.669332,-79.386257,Grocery Store
12,M4Y,0.0,0,43.665860,-79.383160,College Park Area,43.659453,-79.383785,Park
12,M4Y,0.0,0,43.665860,-79.383160,Kinton Ramen,43.660632,-79.378880,Ramen Restaurant
12,M4Y,0.0,0,43.665860,-79.383160,Lebanon Express,43.660991,-79.382976,Falafel Restaurant


In [53]:
Veg_merged = Veg_merged.join(df.set_index('Postal Code'), on='Postal Code')

In [55]:
Veg_merged.shape

(2819, 11)

In [59]:
Veg_merged_map = Veg_merged[Veg_merged["Venue Category"] == 'Vegetarian / Vegan Restaurant']

In [60]:
Veg_merged_map

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
21,M5L,0.020000,1,43.648198,-79.379817,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
20,M5K,0.020000,1,43.647177,-79.381576,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
20,M5K,0.020000,1,43.647177,-79.381576,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
21,M5L,0.020000,1,43.648198,-79.379817,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
30,M6G,0.015625,1,43.669542,-79.422564,Annapurna Vegetarian Restaurant,43.672804,-79.414087,Vegetarian / Vegan Restaurant,Downtown Toronto,Christie
37,M6S,0.014925,1,43.651571,-79.484450,Awai,43.650412,-79.478477,Vegetarian / Vegan Restaurant,West Toronto,"Runnymede, Swansea"
29,M5X,0.020000,1,43.648429,-79.382280,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.020000,1,43.648429,-79.382280,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
9,M4V,0.016393,1,43.686412,-79.400049,nutbar,43.682811,-79.391945,Vegetarian / Vegan Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
0,M4E,0.021277,1,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant,East Toronto,The Beaches


### Map to show clusters of Veg Restaurants in Neighborhood

In [61]:
### Code for Map


Veg_map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(Veg_merged_map['Neighborhood Group Latitude'], Veg_merged_map['Neighborhood Group Longitude'], Veg_merged_map['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(Veg_map_clusters)
       
Veg_map_clusters

In [62]:
#Cluster 0
Veg_merged.loc[(Veg_merged['Cluster Labels'] ==0) & (Veg_merged['Venue Category'] == 'Vegetarian / Vegan Restaurant') ]

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood


In [84]:
# check for neighbourhoods

Recommended_Neighbourhood =0
Recommended_Neighbourhood = Veg_merged.loc[(Veg_merged['Cluster Labels'] ==0) & Veg_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Church and Wellesley', 'The Danforth West, Riverdale',
       'Garden District, Ryerson', 'Studio District',
       'India Bazaar, The Beaches West'],
      dtype='object', name='Neighbourhood')

#### Veg Restauant Recommendation:
Suitable Neighbourhood for Veg restaurant in cluster 0 are : 
'Church and Wellesley', 
'The Danforth West, Riverdale',
'Garden District, Ryerson', 
'Studio District',
'India Bazaar, The Beaches West'

In [86]:
#Cluster 1
Veg_merged.loc[(Veg_merged['Cluster Labels'] ==1) & (Veg_merged['Venue Category'] == 'Vegetarian / Vegan Restaurant') ]

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
21,M5L,0.020000,1,43.648198,-79.379817,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
20,M5K,0.020000,1,43.647177,-79.381576,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
20,M5K,0.020000,1,43.647177,-79.381576,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
21,M5L,0.020000,1,43.648198,-79.379817,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
30,M6G,0.015625,1,43.669542,-79.422564,Annapurna Vegetarian Restaurant,43.672804,-79.414087,Vegetarian / Vegan Restaurant,Downtown Toronto,Christie
37,M6S,0.014925,1,43.651571,-79.484450,Awai,43.650412,-79.478477,Vegetarian / Vegan Restaurant,West Toronto,"Runnymede, Swansea"
29,M5X,0.020000,1,43.648429,-79.382280,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.020000,1,43.648429,-79.382280,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
9,M4V,0.016393,1,43.686412,-79.400049,nutbar,43.682811,-79.391945,Vegetarian / Vegan Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
0,M4E,0.021277,1,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant,East Toronto,The Beaches


#### Analysis
Cluster 1 is having many Veg/Vegan restaurants mainly in : The Beaches, D, avisville North, First Canadian Place, Commerce Court, Toronto Dominion Centre

This cluster is not recommended for a new Veg restaurant

In [89]:
#Cluster 2
Veg_merged.loc[(Veg_merged['Cluster Labels'] ==2) & (Veg_merged['Venue Category'] == 'Vegetarian / Vegan Restaurant') ]

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
26,M5T,0.05,2,43.653206,-79.400049,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.05,2,43.653206,-79.400049,Fresh On Spadina,43.648048,-79.396008,Vegetarian / Vegan Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.05,2,43.653206,-79.400049,Buddha's Vegetarian,43.651904,-79.403312,Vegetarian / Vegan Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.05,2,43.653206,-79.400049,Urban Herbivore,43.656193,-79.402673,Vegetarian / Vegan Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.05,2,43.653206,-79.400049,Hibiscus,43.655454,-79.402439,Vegetarian / Vegan Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
32,M6J,0.04,2,43.647927,-79.419750,Feel Good Guru,43.645488,-79.412575,Vegetarian / Vegan Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.04,2,43.647927,-79.419750,E.L. Ruddy Co. Cafe,43.649532,-79.427828,Vegetarian / Vegan Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.04,2,43.647927,-79.419750,Hello 123,43.643264,-79.423516,Vegetarian / Vegan Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.04,2,43.647927,-79.419750,Veghed,43.649224,-79.422326,Vegetarian / Vegan Restaurant,West Toronto,"Little Portugal, Trinity"


#### Analysis

The cluster 2 is having multiple Veg restaurants in the Neighbourhood : Kensington Market, Chinatown, Grange Park, Little Portugal, Trinity.


In [91]:
#Cluster 3
Veg_merged.loc[(Veg_merged['Cluster Labels'] ==3) & (Veg_merged['Venue Category'] == 'Vegetarian / Vegan Restaurant') ]

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
15,M5C,0.010000,3,43.651494,-79.375418,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,St. James Town
33,M6K,0.010000,3,43.636847,-79.428191,Live Market,43.640044,-79.421433,Vegetarian / Vegan Restaurant,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
38,M7A,0.010000,3,43.662301,-79.389494,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
19,M5J,0.010000,3,43.640816,-79.381752,Kupfert & Kim,43.641179,-79.378144,Vegetarian / Vegan Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
7,M4S,0.012821,3,43.704324,-79.388790,Fresh,43.707324,-79.395649,Vegetarian / Vegan Restaurant,Central Toronto,Davisville
18,M5H,0.010000,3,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
28,M5W,0.010000,3,43.646435,-79.374846,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,Stn A PO Boxes
16,M5E,0.010000,3,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant,Downtown Toronto,Berczy Park
17,M5G,0.010000,3,43.657952,-79.387383,Vegetarian Haven,43.656016,-79.392758,Vegetarian / Vegan Restaurant,Downtown Toronto,Central Bay Street


#### Analysis

The cluster 3 is having multiple Veg restaurants in the Neighbourhood. Not recommended for new restaurant

In [92]:
#Cluster 4
Veg_merged.loc[(Veg_merged['Cluster Labels'] ==4) & (Veg_merged['Venue Category'] == 'Vegetarian / Vegan Restaurant') ]

,Postal Code,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
24,M5R,0.027027,4,43.672710,-79.405678,Annapurna Vegetarian Restaurant,43.672804,-79.414087,Vegetarian / Vegan Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
24,M5R,0.027027,4,43.672710,-79.405678,Fresh on Bloor,43.666755,-79.403491,Vegetarian / Vegan Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
25,M5S,0.030000,4,43.662696,-79.400049,The Hogtown Vegan,43.657007,-79.405302,Vegetarian / Vegan Restaurant,Downtown Toronto,"University of Toronto, Harbord"
25,M5S,0.030000,4,43.662696,-79.400049,Fresh on Bloor,43.666755,-79.403491,Vegetarian / Vegan Restaurant,Downtown Toronto,"University of Toronto, Harbord"
25,M5S,0.030000,4,43.662696,-79.400049,Urban Herbivore,43.656193,-79.402673,Vegetarian / Vegan Restaurant,Downtown Toronto,"University of Toronto, Harbord"


In [93]:
# check for neighbourhoods

Recommended_Neighbourhood =0
Recommended_Neighbourhood = Veg_merged.loc[(Veg_merged['Cluster Labels'] ==4) & Veg_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['The Annex, North Midtown, Yorkville', 'University of Toronto, Harbord'], dtype='object', name='Neighbourhood')

#### Veg Restauant Recommendation:
Suitable Neighbourhood for Veg restaurant in cluster 4 are : University of Toronto, Harbord

## Indian Restaurants

In [210]:
len(venues_grouped[venues_grouped["Indian Restaurant"] > 0])

15

In [211]:
Indian_venues_grouped = venues_grouped[["Postal Code","Indian Restaurant"]]


In [212]:
Indian_venues_grouped.head(12)

,Postal Code,Indian Restaurant
0,M4E,0.021277
1,M4K,0.010000
2,M4L,0.117647
3,M4M,0.000000
4,M4N,0.000000
5,M4P,0.000000
6,M4R,0.000000
7,M4S,0.025641
8,M4T,0.000000
9,M4V,0.000000


### Clustering

In [240]:
from sklearn.cluster import KMeans
n = 5

Indian_cluster = Indian_venues_grouped.drop(["Postal Code"], 1)

# run k-means clustering
Indian_kmeans = KMeans(n_clusters=n, random_state=1)
Indian_kmeans.fit_transform(Indian_cluster)

# check cluster labels generated for each row in the dataframe
Indian_kmeans.labels_[0:20]

array([4, 3, 1, 0, 0, 0, 0, 4, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 3])

In [241]:
Indian_merged = Indian_venues_grouped.copy()

# add clustering labels
Indian_merged["Cluster Labels"] = Indian_kmeans.labels_

In [242]:
Indian_merged.head(5)

,Postal Code,Indian Restaurant,Cluster Labels
0,M4E,0.021277,4
1,M4K,0.010000,3
2,M4L,0.117647,1
3,M4M,0.000000,0
4,M4N,0.000000,0


In [243]:
Indian_merged = Indian_merged.join(all_venues.set_index('Postal Code'), on='Postal Code')

print(Indian_merged.shape)
Indian_merged.head()

(2819, 9)


,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,0.021277,4,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,M4E,0.021277,4,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,M4E,0.021277,4,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,M4E,0.021277,4,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,M4E,0.021277,4,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [244]:
Indian_merged.sort_values(["Cluster Labels"], inplace=True)
Indian_merged.head()

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
20,M5K,0.0,0,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
21,M5L,0.0,0,43.648198,-79.379817,The Fairmont Royal York,43.645449,-79.381508,Hotel
21,M5L,0.0,0,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
21,M5L,0.0,0,43.648198,-79.379817,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
21,M5L,0.0,0,43.648198,-79.379817,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop


In [245]:
Indian_merged = Indian_merged.join(df.set_index('Postal Code'), on='Postal Code')

In [219]:
Indian_merged.count()

Postal Code                     2819
Indian Restaurant               2819
Cluster Labels                  2819
Neighborhood Group Latitude     2819
Neighborhood Group Longitude    2819
Venue                           2819
Venue Latitude                  2819
Venue Longitude                 2819
Venue Category                  2819
Borough                         2819
Neighbourhood                   2819
dtype: int64

In [220]:
Indian_merged.head()

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
20,M5K,0.0,0,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
21,M5L,0.0,0,43.648198,-79.379817,The Fairmont Royal York,43.645449,-79.381508,Hotel,Downtown Toronto,"Commerce Court, Victoria Hotel"
21,M5L,0.0,0,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse,Downtown Toronto,"Commerce Court, Victoria Hotel"
21,M5L,0.0,0,43.648198,-79.379817,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym,Downtown Toronto,"Commerce Court, Victoria Hotel"
21,M5L,0.0,0,43.648198,-79.379817,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop,Downtown Toronto,"Commerce Court, Victoria Hotel"


In [246]:
Indian_merged_map=0
Indian_merged_map = Indian_merged[Indian_merged["Venue Category"] == 'Indian Restaurant']
Indian_merged_map


,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
2,M4L,0.117647,1,43.668999,-79.315572,Udupi Palace,43.672480,-79.321275,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Motimahal,43.672175,-79.322532,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Bombay Chowpatty,43.671875,-79.324114,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Haandi 2000,43.671929,-79.323662,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Gautama,43.672150,-79.322877,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Lahori Taste & Burger House,43.671656,-79.324581,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
30,M6G,0.046875,2,43.669542,-79.422564,Madras Masala,43.662959,-79.421646,Indian Restaurant,Downtown Toronto,Christie
30,M6G,0.046875,2,43.669542,-79.422564,Banjara Indian Cuisine,43.662916,-79.421911,Indian Restaurant,Downtown Toronto,Christie
30,M6G,0.046875,2,43.669542,-79.422564,The Tiffin Box,43.669172,-79.412796,Indian Restaurant,Downtown Toronto,Christie
35,M6P,0.014493,3,43.661608,-79.464763,Curry Twist,43.665581,-79.470541,Indian Restaurant,West Toronto,"High Park, The Junction South"


In [248]:
Indian_map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(Indian_merged_map['Neighborhood Group Latitude'], Indian_merged_map['Neighborhood Group Longitude'], Indian_merged_map['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(Indian_map_clusters)
       
Indian_map_clusters

In [249]:
#Cluster 0
Indian_merged.loc[(Indian_merged['Cluster Labels'] ==0) & (Indian_merged['Venue Category'] == 'Indian Restaurant') ]

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood


In [250]:
# check for neighbourhoods
#Veg_merged.loc[(Veg_merged['Cluster Labels'] ==0) & Veg_merged['Venue Category'].str.contains('Restaurant',case=False)]
#filtered_df = merged_df[merged_df.Borough.str.contains('Toronto',case=False)]
#filtered_df.head()
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Indian_merged.loc[(Indian_merged['Cluster Labels'] ==0) & Indian_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Little Portugal, Trinity', 'Toronto Dominion Centre, Design Exchange',
       'First Canadian Place, Underground city', 'Garden District, Ryerson',
       'Commerce Court, Victoria Hotel'],
      dtype='object', name='Neighbourhood')

#### Analysis and Recommendation

Recommended Neighbourhood for Indian Restaurant in cluser 0 : Little Portugal, Trinity, Toronto Dominion Centre, Design Exchange, Garden District, Ryerson

In [230]:
#Cluster 1
Indian_merged.loc[(Indian_merged['Cluster Labels'] ==1) & (Indian_merged['Venue Category'] == 'Indian Restaurant') ]

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
2,M4L,0.117647,1,43.668999,-79.315572,Udupi Palace,43.672480,-79.321275,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Motimahal,43.672175,-79.322532,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Bombay Chowpatty,43.671875,-79.324114,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Haandi 2000,43.671929,-79.323662,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Gautama,43.672150,-79.322877,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.117647,1,43.668999,-79.315572,Lahori Taste & Burger House,43.671656,-79.324581,Indian Restaurant,East Toronto,"India Bazaar, The Beaches West"


#### Analysis and Recommendation

This cluster is having multiple Indian restaurants . Not recommended to open new Indian restaurant in this Neighbourhood.

In [251]:
#Cluster 2
Indian_merged.loc[(Indian_merged['Cluster Labels'] ==2) & (Indian_merged['Venue Category'] == 'Indian Restaurant') ]

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
30,M6G,0.046875,2,43.669542,-79.422564,Madras Masala,43.662959,-79.421646,Indian Restaurant,Downtown Toronto,Christie
30,M6G,0.046875,2,43.669542,-79.422564,Banjara Indian Cuisine,43.662916,-79.421911,Indian Restaurant,Downtown Toronto,Christie
30,M6G,0.046875,2,43.669542,-79.422564,The Tiffin Box,43.669172,-79.412796,Indian Restaurant,Downtown Toronto,Christie


#### Analysis and Recommendation

This cluser is having multiple Indian restaurants .
But if required Christie can be considered location for Indian Restaurant.

In [252]:
#Cluster 3
Indian_merged.loc[(Indian_merged['Cluster Labels'] ==3) & (Indian_merged['Venue Category'] == 'Indian Restaurant') ]

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
35,M6P,0.014493,3,43.661608,-79.464763,Curry Twist,43.665581,-79.470541,Indian Restaurant,West Toronto,"High Park, The Junction South"
37,M6S,0.014925,3,43.651571,-79.484450,Bukhara indian cuisine,43.651105,-79.477104,Indian Restaurant,West Toronto,"Runnymede, Swansea"
1,M4K,0.010000,3,43.679557,-79.352188,Sher-E-Punjab,43.677308,-79.353066,Indian Restaurant,East Toronto,"The Danforth West, Riverdale"
33,M6K,0.010000,3,43.636847,-79.428191,Mother India,43.640718,-79.436259,Indian Restaurant,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
24,M5R,0.013514,3,43.672710,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
19,M5J,0.010000,3,43.640816,-79.381752,Indian Roti House,43.639060,-79.385422,Indian Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
26,M5T,0.010000,3,43.653206,-79.400049,My Roti Place,43.648663,-79.397306,Indian Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
38,M7A,0.010000,3,43.662301,-79.389494,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
11,M4X,0.015385,3,43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant,Downtown Toronto,"St. James Town, Cabbagetown"


#### Analysis and Recommendation

This cluser is having multiple Indian restaurants . Not recommended to open new Indian restaurant in this Neighbourhood.

In [253]:
#Cluster 4
Indian_merged.loc[(Indian_merged['Cluster Labels'] ==4) & (Indian_merged['Venue Category'] == 'Indian Restaurant') ]

,Postal Code,Indian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
0,M4E,0.021277,4,43.676357,-79.293031,Delhi Bistro,43.672506,-79.288353,Indian Restaurant,East Toronto,The Beaches
12,M4Y,0.020000,4,43.665860,-79.383160,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant,Downtown Toronto,Church and Wellesley
12,M4Y,0.020000,4,43.665860,-79.383160,King's Palace,43.672214,-79.385843,Indian Restaurant,Downtown Toronto,Church and Wellesley
7,M4S,0.025641,4,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant,Central Toronto,Davisville
34,M6N,0.031250,4,43.673185,-79.487262,Roti Time,43.668211,-79.487152,Indian Restaurant,Toronto/York,"Runnymede, The Junction, Weston-Pellam Park, C..."
7,M4S,0.025641,4,43.704324,-79.388790,Banjara Indian Cuisine,43.707810,-79.393296,Indian Restaurant,Central Toronto,Davisville


#### Indian Restauant Recommendation:

Suitable Neighbourhood for Indian restaurant in cluster 4 are : The Beaches , Runnymede, The Junction, Weston-Pellam Park

## Mexicon Restaurants

In [127]:
venues_grouped[venues_grouped["Mexican Restaurant"] > 0]

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.021277,0.0,0.000000,0.000000,0.0,0.00,0.0000,0.0,0.000000
6,M4R,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0000,0.0,0.027027
7,M4S,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.012821,0.0,0.000000,0.000000,0.0,0.00,0.0000,0.0,0.012821
12,M4Y,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.000000,0.0,0.010000,0.000000,0.0,0.00,0.0000,0.0,0.020000
13,M5A,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0125,0.0,0.012500
14,M5B,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.01,0.0000,0.0,0.010000
17,M5G,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.01,0.010000,0.0,0.000000,0.010000,0.0,0.00,0.0000,0.0,0.010000
24,M5R,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.013514,...,0.00,0.027027,0.0,0.000000,0.000000,0.0,0.00,0.0000,0.0,0.000000
25,M5S,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.030000,0.0,0.000000,0.010000,0.0,0.00,0.0000,0.0,0.010000
26,M5T,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.050000,0.0,0.000000,0.020000,0.0,0.01,0.0000,0.0,0.030000


In [128]:
Mexican_venues_grouped = venues_grouped[["Postal Code","Mexican Restaurant"]]

In [129]:
Mexican_venues_grouped.head()

,Postal Code,Mexican Restaurant
0,M4E,0.021277
1,M4K,0.000000
2,M4L,0.000000
3,M4M,0.000000
4,M4N,0.000000


### Clustering

In [130]:
##Clustering

from sklearn.cluster import KMeans
n = 5

Mexican_cluster = Mexican_venues_grouped.drop(["Postal Code"], 1)

# run k-means clustering
Mexican_kmeans = KMeans(n_clusters=n, random_state=1)
Mexican_kmeans.fit_transform(Mexican_cluster)

# check cluster labels generated for each row in the dataframe
Mexican_kmeans.labels_[0:20]

array([4, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 2, 3, 2, 0, 0, 2, 0, 0])

In [131]:
Mexican_merged = Mexican_venues_grouped.copy()

# add clustering labels
Mexican_merged["Cluster Labels"] = Mexican_kmeans.labels_

In [132]:
Mexican_merged = Mexican_merged.join(all_venues.set_index('Postal Code'), on='Postal Code')

print(Mexican_merged.shape)
Mexican_merged.head()

(2819, 9)


,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,0.021277,4,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,M4E,0.021277,4,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,M4E,0.021277,4,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,M4E,0.021277,4,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,M4E,0.021277,4,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [134]:
Mexican_merged.sort_values(["Cluster Labels"], inplace=True)
Mexican_merged.head()

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
20,M5K,0.0,0,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
21,M5L,0.0,0,43.648198,-79.379817,The Fairmont Royal York,43.645449,-79.381508,Hotel
21,M5L,0.0,0,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
21,M5L,0.0,0,43.648198,-79.379817,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
21,M5L,0.0,0,43.648198,-79.379817,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop


In [135]:
Mexican_merged = Mexican_merged.join(df.set_index('Postal Code'), on='Postal Code')

In [136]:
Mexican_merged_map = Mexican_merged[Mexican_merged["Venue Category"] == 'Mexican Restaurant']

In [137]:
Mexican_merged_map

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
26,M5T,0.030000,1,43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.030000,1,43.653206,-79.400049,Torteria San Cosme,43.654702,-79.400646,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.030000,1,43.653206,-79.400049,El Trompo,43.655832,-79.402561,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
24,M5R,0.027027,1,43.672710,-79.405678,Playa Cabana,43.676112,-79.401279,Mexican Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
35,M6P,0.028986,1,43.661608,-79.464763,La Revolucion,43.665460,-79.464308,Mexican Restaurant,West Toronto,"High Park, The Junction South"
35,M6P,0.028986,1,43.661608,-79.464763,Playa Cabana Cantina,43.665315,-79.465548,Mexican Restaurant,West Toronto,"High Park, The Junction South"
6,M4R,0.027027,1,43.715383,-79.405678,Tio's Urban Mexican,43.714630,-79.400000,Mexican Restaurant,Central Toronto,"North Toronto West, Lawrence Park"
24,M5R,0.027027,1,43.672710,-79.405678,Playa Cabana Hacienda,43.677162,-79.398315,Mexican Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
17,M5G,0.010000,2,43.657952,-79.387383,Chipotle Mexican Grill,43.656860,-79.380910,Mexican Restaurant,Downtown Toronto,Central Bay Street
33,M6K,0.010000,2,43.636847,-79.428191,Grand Electric,43.641714,-79.431545,Mexican Restaurant,West Toronto,"Brockton, Parkdale Village, Exhibition Place"


In [138]:
### Code for Map


Mexican_map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(Mexican_merged_map['Neighborhood Group Latitude'], Mexican_merged_map['Neighborhood Group Longitude'], Mexican_merged_map['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(Mexican_map_clusters)
       
Mexican_map_clusters

In [139]:
#Cluster 0
Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==0) & (Mexican_merged['Venue Category'] == 'Mexican Restaurant') ]

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood


In [145]:
# check for neighbourhoods
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==0) & Mexican_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Little Portugal, Trinity', 'Toronto Dominion Centre, Design Exchange',
       'The Danforth West, Riverdale',
       'First Canadian Place, Underground city',
       'Commerce Court, Victoria Hotel'],
      dtype='object', name='Neighbourhood')

#### Veg Restauant Recommendation:

Suitable Neighbourhood for Mexicon restaurant in cluster 0 are : 
'Little Portugal, Trinity', 
'Toronto Dominion Centre, Design Exchange',
'The Danforth West, Riverdale',
'First Canadian Place, Underground city',
'Commerce Court, Victoria Hotel'

In [146]:
#Cluster 1
Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==1) & (Mexican_merged['Venue Category'] == 'Mexican Restaurant') ]

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
26,M5T,0.030000,1,43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.030000,1,43.653206,-79.400049,Torteria San Cosme,43.654702,-79.400646,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
26,M5T,0.030000,1,43.653206,-79.400049,El Trompo,43.655832,-79.402561,Mexican Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
24,M5R,0.027027,1,43.672710,-79.405678,Playa Cabana,43.676112,-79.401279,Mexican Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"
35,M6P,0.028986,1,43.661608,-79.464763,La Revolucion,43.665460,-79.464308,Mexican Restaurant,West Toronto,"High Park, The Junction South"
35,M6P,0.028986,1,43.661608,-79.464763,Playa Cabana Cantina,43.665315,-79.465548,Mexican Restaurant,West Toronto,"High Park, The Junction South"
6,M4R,0.027027,1,43.715383,-79.405678,Tio's Urban Mexican,43.714630,-79.400000,Mexican Restaurant,Central Toronto,"North Toronto West, Lawrence Park"
24,M5R,0.027027,1,43.672710,-79.405678,Playa Cabana Hacienda,43.677162,-79.398315,Mexican Restaurant,Central Toronto,"The Annex, North Midtown, Yorkville"


#### Analysis
Cluster 1 is having many Mexican restaurants near by.

This cluster is not recommended for a new Mexican restaurant

In [147]:
#Cluster 2
Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==2) & (Mexican_merged['Venue Category'] == 'Mexican Restaurant') ]

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
17,M5G,0.01,2,43.657952,-79.387383,Chipotle Mexican Grill,43.656860,-79.380910,Mexican Restaurant,Downtown Toronto,Central Bay Street
33,M6K,0.01,2,43.636847,-79.428191,Grand Electric,43.641714,-79.431545,Mexican Restaurant,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
25,M5S,0.01,2,43.662696,-79.400049,El Trompo,43.655832,-79.402561,Mexican Restaurant,Downtown Toronto,"University of Toronto, Harbord"
12,M4Y,0.01,2,43.665860,-79.383160,Como En Casa,43.665160,-79.384796,Mexican Restaurant,Downtown Toronto,Church and Wellesley
38,M7A,0.01,2,43.662301,-79.389494,Como En Casa,43.665160,-79.384796,Mexican Restaurant,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
14,M5B,0.01,2,43.657162,-79.378937,Chipotle Mexican Grill,43.656860,-79.380910,Mexican Restaurant,Downtown Toronto,"Garden District, Ryerson"


#### Analysis
Cluster 2 is having many Mexican restaurants near by.

This cluster is not recommended for a new Mexican restaurant

In [149]:
#Cluster 3
Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==3) & (Mexican_merged['Venue Category'] == 'Mexican Restaurant') ]

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
13,M5A,0.012500,3,43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant,Downtown Toronto,"Regent Park, Harbourfront"
7,M4S,0.012821,3,43.704324,-79.388790,Chacho's,43.697808,-79.389126,Mexican Restaurant,Central Toronto,Davisville
30,M6G,0.015625,3,43.669542,-79.422564,El Pocho,43.669589,-79.413423,Mexican Restaurant,Downtown Toronto,Christie


In [150]:
# check for neighbourhoods
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==3) & Mexican_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Davisville', 'Christie', 'Regent Park, Harbourfront'], dtype='object', name='Neighbourhood')

#### Veg Restauant Recommendation:

Suitable Neighbourhood for Mexicon restaurant in cluster 3 are : 'Christie'
'

In [151]:
#Cluster 4
Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==4) & (Mexican_merged['Venue Category'] == 'Mexican Restaurant') ]

,Postal Code,Mexican Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
0,M4E,0.021277,4,43.676357,-79.293031,Xola,43.672603,-79.28808,Mexican Restaurant,East Toronto,The Beaches


In [152]:
# check for neighbourhoods
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Mexican_merged.loc[(Mexican_merged['Cluster Labels'] ==4) & Mexican_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['The Beaches'], dtype='object', name='Neighbourhood')

## Japanese Restaurants

In [153]:
venues_grouped[venues_grouped["Japanese Restaurant"] > 0]

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.021277,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000
5,M4P,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.017857,0.0,0.00,0.000000,0.00,0.017857,0.0,0.0,0.000000
7,M4S,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.012821,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.012821
8,M4T,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000
9,M4V,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,...,0.00,0.016393,0.0,0.00,0.016393,0.00,0.000000,0.0,0.0,0.016393
11,M4X,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000
12,M4Y,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.000000,0.0,0.01,0.000000,0.00,0.000000,0.0,0.0,0.020000
14,M5B,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.000000,0.0,0.00,0.000000,0.00,0.010000,0.0,0.0,0.010000
15,M5C,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,...,0.00,0.010000,0.0,0.00,0.000000,0.00,0.010000,0.0,0.0,0.000000
16,M5E,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.010000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.000000


In [156]:
Japanese_venues_grouped = venues_grouped[["Postal Code","Japanese Restaurant"]]

In [157]:
Japanese_venues_grouped.head()

,Postal Code,Japanese Restaurant
0,M4E,0.042553
1,M4K,0.000000
2,M4L,0.000000
3,M4M,0.000000
4,M4N,0.000000


### Clustering

In [158]:
from sklearn.cluster import KMeans
n = 5

Japanese_cluster = Japanese_venues_grouped.drop(["Postal Code"], 1)

# run k-means clustering
Japanese_kmeans = KMeans(n_clusters=n, random_state=1)
Japanese_kmeans.fit_transform(Japanese_cluster)

# check cluster labels generated for each row in the dataframe
Japanese_kmeans.labels_[0:20]

array([4, 0, 0, 0, 0, 3, 0, 3, 2, 3, 0, 1, 4, 0, 4, 3, 4, 4, 1, 1])

In [159]:
Japanese_merged = Japanese_venues_grouped.copy()

# add clustering labels
Japanese_merged["Cluster Labels"] = Japanese_kmeans.labels_

In [160]:
Japanese_merged.count()

Postal Code            40
Japanese Restaurant    40
Cluster Labels         40
dtype: int64

In [161]:
Japanese_merged = Japanese_merged.join(all_venues.set_index('Postal Code'), on='Postal Code')

print(Japanese_merged.shape)
Japanese_merged.head()

(2819, 9)


,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,0.042553,4,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,M4E,0.042553,4,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,M4E,0.042553,4,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,M4E,0.042553,4,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,M4E,0.042553,4,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [162]:
Japanese_merged.sort_values(["Cluster Labels"], inplace=True)
Japanese_merged.head()

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
39,M7Y,0.0,0,43.662744,-79.321558,Lakeshore Blvd E & Leslie St,43.658763,-79.328988,Intersection
36,M6R,0.0,0,43.648960,-79.456325,The Ace,43.646144,-79.448860,American Restaurant
36,M6R,0.0,0,43.648960,-79.456325,Scout,43.650970,-79.450866,Gift Shop
36,M6R,0.0,0,43.648960,-79.456325,Amphitheatre - High Park,43.646293,-79.462346,Amphitheater
36,M6R,0.0,0,43.648960,-79.456325,Reunion Island Coffee Bar,43.650463,-79.450610,Coffee Shop


In [163]:
Japanese_merged = Japanese_merged.join(df.set_index('Postal Code'), on='Postal Code')

In [164]:
Japanese_merged_map = Japanese_merged[Japanese_merged["Venue Category"] == 'Japanese Restaurant']

In [165]:
Japanese_merged_map

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
29,M5X,0.030000,1,43.648429,-79.382280,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.030000,1,43.648429,-79.382280,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
21,M5L,0.030000,1,43.648198,-79.379817,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
29,M5X,0.030000,1,43.648429,-79.382280,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
21,M5L,0.030000,1,43.648198,-79.379817,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
19,M5J,0.030000,1,43.640816,-79.381752,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
19,M5J,0.030000,1,43.640816,-79.381752,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
19,M5J,0.030000,1,43.640816,-79.381752,Miku,43.641374,-79.377531,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
18,M5H,0.030000,1,43.650571,-79.384568,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
18,M5H,0.030000,1,43.650571,-79.384568,Manpuku まんぷく,43.653612,-79.390613,Japanese Restaurant,Downtown Toronto,"Richmond, Adelaide, King"


In [166]:
### Code for Map


Japanese_map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(Japanese_merged_map['Neighborhood Group Latitude'], Japanese_merged_map['Neighborhood Group Longitude'], Japanese_merged_map['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(Japanese_map_clusters)
       
Japanese_map_clusters

In [167]:
#Cluster 0
Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==0) & (Japanese_merged['Venue Category'] == 'Japanese Restaurant') ]

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood


In [168]:
# check for neighbourhoods
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==0) & Japanese_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['The Danforth West, Riverdale', 'Studio District',
       'India Bazaar, The Beaches West', 'Runnymede, Swansea',
       'Parkdale, Roncesvalles'],
      dtype='object', name='Neighbourhood')

#### Japanese Restauant Recommendation:
Suitable Neighbourhood for Japanese restaurant in cluster 0 are : 
'The Danforth West, Riverdale', 'Studio District',
'Runnymede, Swansea',
'Parkdale, Roncesvalles'

In [169]:
#Cluster 1
Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==1) & (Japanese_merged['Venue Category'] == 'Japanese Restaurant') ]

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
29,M5X,0.030000,1,43.648429,-79.382280,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.030000,1,43.648429,-79.382280,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
21,M5L,0.030000,1,43.648198,-79.379817,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
29,M5X,0.030000,1,43.648429,-79.382280,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
21,M5L,0.030000,1,43.648198,-79.379817,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Commerce Court, Victoria Hotel"
19,M5J,0.030000,1,43.640816,-79.381752,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
19,M5J,0.030000,1,43.640816,-79.381752,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
19,M5J,0.030000,1,43.640816,-79.381752,Miku,43.641374,-79.377531,Japanese Restaurant,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands"
18,M5H,0.030000,1,43.650571,-79.384568,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
18,M5H,0.030000,1,43.650571,-79.384568,Manpuku まんぷく,43.653612,-79.390613,Japanese Restaurant,Downtown Toronto,"Richmond, Adelaide, King"


#### Analysis
Cluster 1 is having many Japanese restaurants.
This cluster is not recommended for a new Japanese restaurant

In [170]:
#Cluster 2
Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==2) & (Japanese_merged['Venue Category'] == 'Japanese Restaurant') ]

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
8,M4T,0.076923,2,43.689574,-79.383160,Suki Japanese Cuisine,43.688668,-79.391854,Japanese Restaurant,Central Toronto,"Moore Park, Summerhill East"
23,M5P,0.066667,2,43.696948,-79.411307,EDO,43.703754,-79.412802,Japanese Restaurant,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"


In [171]:
# check for neighbourhoods
Recommended_Neighbourhood =0
Recommended_Neighbourhood = Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==2) & Japanese_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Forest Hill North & West, Forest Hill Road Park', 'Moore Park, Summerhill East'], dtype='object', name='Neighbourhood')

#### Japanese Restauant Recommendation:
Suitable Neighbourhood for Japanese restaurant in cluster 2 are : Moore Park, Summerhill East'

In [172]:
#Cluster 3
Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==3) & (Japanese_merged['Venue Category'] == 'Japanese Restaurant') ]

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
26,M5T,0.010000,3,43.653206,-79.400049,Gushi,43.652258,-79.404884,Japanese Restaurant,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
33,M6K,0.010000,3,43.636847,-79.428191,Guu Izakaya,43.641846,-79.431086,Japanese Restaurant,West Toronto,"Brockton, Parkdale Village, Exhibition Place"
30,M6G,0.015625,3,43.669542,-79.422564,Japanhako,43.663811,-79.417655,Japanese Restaurant,Downtown Toronto,Christie
32,M6J,0.020000,3,43.647927,-79.419750,Imanishi Japanese Kitchen,43.649546,-79.426551,Japanese Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.020000,3,43.647927,-79.419750,Bazara,43.648535,-79.420521,Japanese Restaurant,West Toronto,"Little Portugal, Trinity"
20,M5K,0.020000,3,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,"Toronto Dominion Centre, Design Exchange"
15,M5C,0.020000,3,43.651494,-79.375418,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant,Downtown Toronto,St. James Town
9,M4V,0.016393,3,43.686412,-79.400049,Suki Japanese Cuisine,43.688668,-79.391854,Japanese Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
15,M5C,0.020000,3,43.651494,-79.375418,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,St. James Town
5,M4P,0.017857,3,43.712751,-79.390197,Oisie sushi,43.708301,-79.388424,Japanese Restaurant,Central Toronto,Davisville North


#### Analysis
Cluster 3 is having many Japanese restaurants.
This cluster is not recommended for a new Japanese restaurant

In [173]:
#Cluster 4
Japanese_merged.loc[(Japanese_merged['Cluster Labels'] ==4) & (Japanese_merged['Venue Category'] == 'Japanese Restaurant') ]

,Postal Code,Japanese Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
12,M4Y,0.050000,4,43.665860,-79.383160,Tokyo Grill,43.665085,-79.384707,Japanese Restaurant,Downtown Toronto,Church and Wellesley
12,M4Y,0.050000,4,43.665860,-79.383160,Kawa Sushi,43.663894,-79.380210,Japanese Restaurant,Downtown Toronto,Church and Wellesley
12,M4Y,0.050000,4,43.665860,-79.383160,Tokyo Kitchen,43.668783,-79.385153,Japanese Restaurant,Downtown Toronto,Church and Wellesley
12,M4Y,0.050000,4,43.665860,-79.383160,Kinka Izakaya Original,43.660596,-79.378891,Japanese Restaurant,Downtown Toronto,Church and Wellesley
12,M4Y,0.050000,4,43.665860,-79.383160,Onnki Donburi,43.669757,-79.384574,Japanese Restaurant,Downtown Toronto,Church and Wellesley
0,M4E,0.042553,4,43.676357,-79.293031,Yumei Sushi,43.671108,-79.295064,Japanese Restaurant,East Toronto,The Beaches
0,M4E,0.042553,4,43.676357,-79.293031,Bikkuri Japanese Restaurant,43.670715,-79.296956,Japanese Restaurant,East Toronto,The Beaches
16,M5E,0.040000,4,43.644771,-79.373306,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant,Downtown Toronto,Berczy Park
16,M5E,0.040000,4,43.644771,-79.373306,Chotto Matte,43.646473,-79.378782,Japanese Restaurant,Downtown Toronto,Berczy Park
16,M5E,0.040000,4,43.644771,-79.373306,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant,Downtown Toronto,Berczy Park


#### Analysis
Cluster 4 is having many Japanese restaurants.
This cluster is not recommended for a new Japanese restaurant

## Italian Restaurants

In [174]:
venues_grouped[venues_grouped["Italian Restaurant"] > 0]

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
1,M4K,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000
2,M4L,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.019608,0.000000
3,M4M,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.032258,...,0.00,0.000000,0.0,0.000000,0.021505,0.00,0.000000,0.000000,0.000000,0.021505
5,M4P,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.017857,0.0,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000
6,M4R,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.027027
7,M4S,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.012821,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012821
9,M4V,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,...,0.00,0.016393,0.0,0.000000,0.016393,0.00,0.000000,0.000000,0.000000,0.016393
11,M4X,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
12,M4Y,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,...,0.00,0.000000,0.0,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000
13,M5A,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.012500,0.000000,0.012500


In [176]:
Italian_venues_grouped = venues_grouped[["Postal Code","Italian Restaurant"]]

In [177]:
Italian_venues_grouped.head()

,Postal Code,Italian Restaurant
0,M4E,0.000000
1,M4K,0.030000
2,M4L,0.039216
3,M4M,0.021505
4,M4N,0.000000


### Clustering

In [178]:
from sklearn.cluster import KMeans
n = 5

Italian_cluster = Italian_venues_grouped.drop(["Postal Code"], 1)

# run k-means clustering
Italian_kmeans = KMeans(n_clusters=n, random_state=1)
Italian_kmeans.fit_transform(Italian_cluster)

# check cluster labels generated for each row in the dataframe
Italian_kmeans.labels_[0:20]

array([3, 4, 4, 0, 3, 2, 2, 1, 3, 2, 3, 4, 4, 4, 4, 4, 0, 0, 0, 0])

In [179]:
Italian_merged = Italian_venues_grouped.copy()

# add clustering labels
Italian_merged["Cluster Labels"] = Italian_kmeans.labels_

In [180]:
Italian_merged

,Postal Code,Italian Restaurant,Cluster Labels
0,M4E,0.000000,3
1,M4K,0.030000,4
2,M4L,0.039216,4
3,M4M,0.021505,0
4,M4N,0.000000,3
5,M4P,0.071429,2
6,M4R,0.054054,2
7,M4S,0.102564,1
8,M4T,0.000000,3
9,M4V,0.065574,2


In [181]:
Italian_merged = Italian_merged.join(all_venues.set_index('Postal Code'), on='Postal Code')

print(Veg_merged.shape)
Italian_merged.head()

(2819, 11)


,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,0.0,3,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,M4E,0.0,3,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
0,M4E,0.0,3,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
0,M4E,0.0,3,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
0,M4E,0.0,3,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [183]:
Italian_merged.sort_values(["Cluster Labels"], inplace=True)
Italian_merged.head()

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
20,M5K,0.02,0,43.647177,-79.381576,Ki Modern Japanese + Bar,43.647223,-79.379374,Japanese Restaurant
28,M5W,0.01,0,43.646435,-79.374846,Pilot Coffee Roasters,43.648835,-79.380936,Coffee Shop
28,M5W,0.01,0,43.646435,-79.374846,The Poet Cafe,43.650637,-79.371276,Café
28,M5W,0.01,0,43.646435,-79.374846,Terroni,43.650927,-79.375602,Italian Restaurant
28,M5W,0.01,0,43.646435,-79.374846,Rexall,43.648182,-79.373870,Pharmacy


In [184]:
Italian_merged = Italian_merged.join(df.set_index('Postal Code'), on='Postal Code')

In [185]:
Italian_merged_map = Italian_merged[Italian_merged["Venue Category"] == 'Italian Restaurant']

In [186]:
Italian_merged_map

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
28,M5W,0.010000,0,43.646435,-79.374846,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,Stn A PO Boxes
18,M5H,0.020000,0,43.650571,-79.384568,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
29,M5X,0.020000,0,43.648429,-79.382280,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.020000,0,43.648429,-79.382280,TOCA,43.645431,-79.387059,Italian Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
30,M6G,0.015625,0,43.669542,-79.422564,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant,Downtown Toronto,Christie
...,...,...,...,...,...,...,...,...,...,...,...
11,M4X,0.030769,4,43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant,Downtown Toronto,"St. James Town, Cabbagetown"
13,M5A,0.025000,4,43.654260,-79.360636,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant,Downtown Toronto,"Regent Park, Harbourfront"
13,M5A,0.025000,4,43.654260,-79.360636,Fusaro's,43.653347,-79.369517,Italian Restaurant,Downtown Toronto,"Regent Park, Harbourfront"
12,M4Y,0.030000,4,43.665860,-79.383160,Blu Ristorante and Lounge,43.671685,-79.388614,Italian Restaurant,Downtown Toronto,Church and Wellesley


In [187]:
### Code for Map


Italian_map_clusters = folium.Map(location=[latitude, longitude],zoom_start=14)

# set color scheme for the clusters


# add markers to the map
markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(Italian_merged_map['Neighborhood Group Latitude'], Italian_merged_map['Neighborhood Group Longitude'], Italian_merged_map['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(Italian_map_clusters)
       
Italian_map_clusters

In [188]:
#Cluster 0
Italian_merged.loc[(Italian_merged['Cluster Labels'] ==0) & (Italian_merged['Venue Category'] == 'Italian Restaurant') ]

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
28,M5W,0.010000,0,43.646435,-79.374846,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,Stn A PO Boxes
18,M5H,0.020000,0,43.650571,-79.384568,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
29,M5X,0.020000,0,43.648429,-79.382280,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
29,M5X,0.020000,0,43.648429,-79.382280,TOCA,43.645431,-79.387059,Italian Restaurant,Downtown Toronto,"First Canadian Place, Underground city"
30,M6G,0.015625,0,43.669542,-79.422564,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant,Downtown Toronto,Christie
16,M5E,0.020000,0,43.644771,-79.373306,Terroni,43.650927,-79.375602,Italian Restaurant,Downtown Toronto,Berczy Park
16,M5E,0.020000,0,43.644771,-79.373306,Carisma,43.649617,-79.375434,Italian Restaurant,Downtown Toronto,Berczy Park
17,M5G,0.010000,0,43.657952,-79.387383,Mercatto,43.660391,-79.387664,Italian Restaurant,Downtown Toronto,Central Bay Street
18,M5H,0.020000,0,43.650571,-79.384568,TOCA,43.645431,-79.387059,Italian Restaurant,Downtown Toronto,"Richmond, Adelaide, King"
3,M4M,0.021505,0,43.659526,-79.340923,Baldini,43.661300,-79.339027,Italian Restaurant,East Toronto,Studio District


#### Analysis
Cluster 0 is having many Italian restaurants near by.

This cluster is not recommended for a new Italian restaurant

In [189]:
#Cluster 1
Italian_merged.loc[(Italian_merged['Cluster Labels'] ==1) & (Italian_merged['Venue Category'] == 'Italian Restaurant') ]

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
23,M5P,0.133333,1,43.696948,-79.411307,7 Numbers,43.703630,-79.413724,Italian Restaurant,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
23,M5P,0.133333,1,43.696948,-79.411307,Ferraro,43.703655,-79.413167,Italian Restaurant,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
7,M4S,0.102564,1,43.704324,-79.388790,Grano,43.701712,-79.397318,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Balsamico,43.701505,-79.397162,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Bar Buca,43.706961,-79.394808,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Positano,43.704558,-79.388639,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Five Doors North,43.702236,-79.397526,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Zucca Trattoria,43.703777,-79.397824,Italian Restaurant,Central Toronto,Davisville
7,M4S,0.102564,1,43.704324,-79.388790,Il Sogno,43.700520,-79.396944,Italian Restaurant,Central Toronto,Davisville


In [195]:
# check for neighbourhoods

Recommended_Neighbourhood =0
Recommended_Neighbourhood = Italian_merged.loc[(Italian_merged['Cluster Labels'] ==1) & Italian_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Davisville', 'Forest Hill North & West, Forest Hill Road Park'], dtype='object', name='Neighbourhood')

#### Italian Restauant Recommendation:
Suitable Neighbourhood for Italian restaurant in cluster 1 are : Forest Hill North & West, Forest Hill Road Park


In [190]:
#Cluster 2
Italian_merged.loc[(Italian_merged['Cluster Labels'] ==2) & (Italian_merged['Venue Category'] == 'Italian Restaurant') ]

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
9,M4V,0.065574,2,43.686412,-79.400049,Nove Trattoria,43.686795,-79.393609,Italian Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
9,M4V,0.065574,2,43.686412,-79.400049,Boccone Deli Cafe & Pizza,43.686421,-79.393443,Italian Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
9,M4V,0.065574,2,43.686412,-79.400049,Capocaccia Café,43.685915,-79.393305,Italian Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
9,M4V,0.065574,2,43.686412,-79.400049,Spiga,43.686393,-79.393354,Italian Restaurant,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
5,M4P,0.071429,2,43.712751,-79.390197,Bar Buca,43.706961,-79.394808,Italian Restaurant,Central Toronto,Davisville North
5,M4P,0.071429,2,43.712751,-79.390197,La Vecchia Ristorante,43.710167,-79.399086,Italian Restaurant,Central Toronto,Davisville North
5,M4P,0.071429,2,43.712751,-79.390197,Grazie Ristorante,43.709329,-79.398823,Italian Restaurant,Central Toronto,Davisville North
5,M4P,0.071429,2,43.712751,-79.390197,Cibo Wine Bar,43.711464,-79.399570,Italian Restaurant,Central Toronto,Davisville North
6,M4R,0.054054,2,43.715383,-79.405678,Cibo Wine Bar,43.711464,-79.399570,Italian Restaurant,Central Toronto,"North Toronto West, Lawrence Park"
6,M4R,0.054054,2,43.715383,-79.405678,La Vecchia Ristorante,43.710167,-79.399086,Italian Restaurant,Central Toronto,"North Toronto West, Lawrence Park"


#### Analysis
Cluster 2 is having many Italian restaurants near by.

This cluster is not recommended for a new Italian restaurant

In [191]:
#Cluster 3
Italian_merged.loc[(Italian_merged['Cluster Labels'] ==3) & (Italian_merged['Venue Category'] == 'Italian Restaurant') ]

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood


In [194]:
# check for neighbourhoods

Recommended_Neighbourhood =0
Recommended_Neighbourhood = Italian_merged.loc[(Italian_merged['Cluster Labels'] ==3) & Italian_merged['Venue Category'].str.contains('Restaurant',case=False)].groupby('Neighbourhood').count().sort_values(by='Postal Code', ascending=False).head()

Recommended_Neighbourhood.index

Index(['Kensington Market, Chinatown, Grange Park', 'The Beaches',
       'Runnymede, The Junction, Weston-Pellam Park, Carlton Village',
       'Moore Park, Summerhill East', 'Lawrence Park'],
      dtype='object', name='Neighbourhood')

#### Italian Restauant Recommendation:
Suitable Neighbourhood for Italian restaurant in cluster 3 are : 
'Kensington Market, Chinatown, Grange Park', 
'The Beaches',
'Runnymede, The Junction, Weston-Pellam Park, Carlton Village',
'Moore Park, Summerhill East' 
       

In [192]:
#Cluster 4
Italian_merged.loc[(Italian_merged['Cluster Labels'] ==4) & (Italian_merged['Venue Category'] == 'Italian Restaurant') ]

,Postal Code,Italian Restaurant,Cluster Labels,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,Neighbourhood
1,M4K,0.030000,4,43.679557,-79.352188,IL FORNELLO on Danforth,43.678604,-79.346904,Italian Restaurant,East Toronto,"The Danforth West, Riverdale"
35,M6P,0.043478,4,43.661608,-79.464763,nodo,43.665303,-79.465621,Italian Restaurant,West Toronto,"High Park, The Junction South"
35,M6P,0.043478,4,43.661608,-79.464763,Annette Food Market,43.662457,-79.470979,Italian Restaurant,West Toronto,"High Park, The Junction South"
32,M6J,0.040000,4,43.647927,-79.419750,Noce,43.645550,-79.411294,Italian Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.040000,4,43.647927,-79.419750,Levetto,43.641425,-79.422180,Italian Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.040000,4,43.647927,-79.419750,Enoteca Sociale,43.649535,-79.425792,Italian Restaurant,West Toronto,"Little Portugal, Trinity"
32,M6J,0.040000,4,43.647927,-79.419750,Ufficio,43.649439,-79.423014,Italian Restaurant,West Toronto,"Little Portugal, Trinity"
2,M4L,0.039216,4,43.668999,-79.315572,Casa di Giorgio,43.666645,-79.315204,Italian Restaurant,East Toronto,"India Bazaar, The Beaches West"
2,M4L,0.039216,4,43.668999,-79.315572,Pizzeria Via Mercanti,43.672641,-79.319863,Italian Restaurant,East Toronto,"India Bazaar, The Beaches West"
38,M7A,0.030000,4,43.662301,-79.389494,Dal Moro's Fresh Pasta To Go,43.666641,-79.385400,Italian Restaurant,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Analysis
Cluster 4 is having many Italian restaurants near by.

This cluster is not recommended for a new Italian restaurant

# Recommendation Summary 

Vegetarian / Vegan Restaurant : 
- Church and Wellesley
- The Danforth West, Riverdale
- Garden District, Ryerson'
- Studio District
- India Bazaar, The Beaches West
- University of Toronto, Harbord
 
Indian Restaurant
- The Beaches , Runnymede
- The Junction, Weston- Pellam Park

Mexican Restaurant
- Christie 
- The Beaches

Japanese Restaurant
- Moore Park, Summerhill East' 

Italian Restaurant
- Kensington Market, Chinatown, Grange Park
- The Beaches
- Runnymede, The Junction, Weston- Pellam Park Carlton Village
- Moore Park, Summerhill East
- Forest Hill North & West, Forest Hill Road Park


**Discussion:** 

This project will be very helpful to analyse all the possible data and get best recommendations.
The approach is very useful and can be used for n number of fields like Restaurant business, Construction business, Grocery Outlets, Hospitals etc.

I will be enhancing the project going forward. 

Future Enhancement and Improvement plan:

-	Create a recommender API using the project.
-	Further create a web interface to use the API and show recommendations on basis of the location selected.
-	Enhance the visualizations and clustering techniques.


**Conclusion:**  
This project will help a Restaurant business to analyse and get recommendation for the best suitable neighbourhood and Restaurant types.

During the whole course I was able to learn the data science techniques which can help us to achieve better goals in nearly all the fields of life like Manufacturing, Construction, Hospitality, Healthcare, Automobile etc.

